In [1]:
import time
import datetime
import requests

In [2]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [3]:
api_request_limit_per_hour = 4999
error_url = []
limit_check_hour = datetime.datetime.now().hour
can_continue = True

def HasItPassLimitHour(current_hour):
    return limit_check_hour != current_hour

def CheckRequestLimit():
    global api_request_limit_per_hour
    if api_request_limit_per_hour != 0:
        can_continue = True
    else:
        current_hour = datetime.datetime.now().hour
        if HasItPassLimitHour(current_hour) == True:
            limit_check_hour = datetime.datetime.now().hour
            api_request_limit_per_hour = 4999
            can_continue = True
        else:
            can_continue = False
    
def FetchDataFromGitHubRestApi(url):
    global api_request_limit_per_hour
    CheckRequestLimit()
    while can_continue == False:
        time.sleep(60)
        CheckRequestLimit()
    response = session.get(url)
    api_request_limit_per_hour -= 1
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if field:
        if type(field) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [5]:
username = "kritserv"

In [6]:
profile_url = "https://api.github.com/users/"+username
profile_data = FetchDataFromGitHubRestApi(profile_url)

In [7]:
event_data = {}

for i in (1,2,3):
    event_url_pagination = "https://api.github.com/users/"+username+"/events?per_page=100&page="+str(i)
    event_data["page"+str(i)] = FetchDataFromGitHubRestApi(event_url_pagination)

In [8]:
all_event_data = []
for page in event_data:
    for event in event_data[page]:
        all_event_data.append((Select('type', event), Select('created_at', event)))

In [9]:
len(all_event_data)

299

In [10]:
len(event_data["page1"]), len(event_data["page2"]), len(event_data["page3"])

(100, 100, 99)

In [11]:
for i in range(10):
    print(all_event_data[i])

('PushEvent', '2024-01-11T09:32:07Z')
('PushEvent', '2024-01-11T09:27:29Z')
('PushEvent', '2024-01-10T12:03:32Z')
('PushEvent', '2024-01-10T11:46:46Z')
('PushEvent', '2024-01-10T10:46:18Z')
('PushEvent', '2024-01-10T10:43:03Z')
('PushEvent', '2024-01-10T10:40:22Z')
('PushEvent', '2024-01-09T10:23:54Z')
('PushEvent', '2024-01-08T13:14:14Z')
('PushEvent', '2024-01-08T10:17:47Z')


In [12]:
api_request_limit_per_hour

4995

In [13]:
starred_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username+"/starred?per_page=100&sort=updated")

In [14]:
len(starred_data)

36

In [15]:
for star_repo in starred_data:
    print("full name: "+Select("full_name", star_repo), 
          "html_url: "+Select("html_url", star_repo), 
          "description: "+Select("description", star_repo), 
          "language: "+Select("language", star_repo), 
          "topics: "+Select("topics", star_repo), 
          "stargazers_count: "+Select("stargazers_count", star_repo), sep='\n', end='\n\n')

full name: postgres/postgres
html_url: https://github.com/postgres/postgres
description: Mirror of the official PostgreSQL GIT repository. Note that this is just a *mirror* - we dont work with pull requests on github. To contribute/please see https://wiki.postgresql.org/wiki/Submitting_a_Patch
language: C
topics: None
stargazers_count: 13873

full name: mongodb/mongo
html_url: https://github.com/mongodb/mongo
description: The MongoDB Database
language: C++
topics: c-plus-plus/database/mongodb/nosql
stargazers_count: 24921

full name: 4ian/GDevelop
html_url: https://github.com/4ian/GDevelop
description: :video_game: Open-source/cross-platform game engine designed to be used by everyone.
language: JavaScript
topics: 2d-game/2d-game-engine/3d/3d-game/3d-game-engine/c-plus-plus/game/game-2d/game-3d/game-development/game-engine/gamedev/gamemaker/gdevelop/hacktoberfest/html5/html5-game-engine/javascript/open-source/web-app
stargazers_count: 5536

full name: django/django
html_url: https://gi

In [16]:
print('login: '+Select("login", profile_data), 
      'type: '+Select("type", profile_data), 
      'location: '+Select("location", profile_data), 
      'hireable: '+Select("hireable", profile_data), 
      'public_repos: '+Select("public_repos", profile_data), 
      'public_gists: '+Select("public_gists", profile_data), 
      'followers: '+Select("followers", profile_data), 
      'following: '+Select("following", profile_data), 
      'created_at: '+Select("created_at", profile_data), 
      'updated_at: '+Select("updated_at", profile_data), sep='\n')

login: kritserv
type: User
location: Bangkok/Thailand
hireable: True
public_repos: 14
public_gists: 1
followers: 3
following: 5
created_at: 2023-01-10T12:07:55Z
updated_at: 2024-01-06T14:57:09Z


In [17]:
repo_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username+"/repos?per_page=100&sort=updated")

In [18]:
for repo in repo_data:
    print("description: "+Select("description", repo), 
          "fork: "+Select("fork", repo), 
          "stargazers_count: "+Select("stargazers_count", repo), 
          "language: "+Select("language", repo), 
          "topics: "+Select("topics", repo), 
          "forks: "+Select("forks", repo), sep='\n', end='\n\n')

description: API for the tarot cards of the Rider-Waite-Smith deck using go
fork: True
stargazers_count: 0
language: Go
topics: api/go/golang/tarot-cards
forks: 0

description: A python code that prints the result of a neural network trying to solve Pythagoras’ theorem in real-time in your terminal.
fork: False
stargazers_count: 0
language: Python
topics: neural-network/pythagoras/python
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: None
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: None
forks: 0

description: None
fork: False
stargazers_count: 0
language: HTML
topics: None
forks: 0

description: Ecommerce website build with Django/Docker/SQLite.
fork: False
stargazers_count: 0
language: Python
topics: django/django-project/docker/ecommerce-website/python
forks: 0

description: None
fork: False
stargazers_count: 0
language: Python
topics: None
forks: 0

description: A python notebook for fake and real voice (from

In [19]:
api_request_limit_per_hour

4993